# Mongo

In [1]:
import pymongo
import json
import os
from bson import ObjectId
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
# String de conexion a mongo en local
CONN_STRING = "mongodb://127.0.0.1:27017/?readPreference=primary&serverSelectionTimeoutMS=2000&directConnection=true&ssl=false"

In [3]:
# creamos un cliente para interactuar con mongo
myclient = pymongo.MongoClient(CONN_STRING)

# 0. Uploading all jsons to Mongo

In [4]:
# conexion a la base edca.raw para colocar los json descargados
db_raw = myclient["edca"]["raw"]

In [5]:
myclient.close()

In [6]:
df = pd.DataFrame({"a": range(10), "b:": [1 for _ in range(10)]})

In [7]:
df.to_csv("write_test.csv.zip", index=False,compression='zip')

In [8]:
# Nombre de los archivos json en la carpeta de trabajo
path_to_json = './contratacionesabiertas_bulk_paquetes_json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [9]:
for i in tqdm(range(1,99)):

    json_files = f'contratacionesabiertas_bulk_paquete{i}.json'
    # print('Uploading',json_files)

    # Lee el contenido del archivo json como diccionatio
    with open('./contratacionesabiertas_bulk_paquetes_json/'+json_files) as json_file:

        data = json.load(json_file)
    
    for x in data:
        x['_id'] = ObjectId(x['_id']['$oid'])
        x['paquete'] = i
        
    db_raw.insert_many(data)

# 1. Querys en mongo

## 1.1 Informacion general de la licitacion

In [8]:
def get_query_generals(i: int)-> list:
    """
    Obtain the general pipeline query for i-th package
    """

    query_generals = [
        {
        "$match": {"paquete": i}
        },
        {
            '$unwind': {
                'path': '$releases', 
                'includeArrayIndex': 'release_index', 
                'preserveNullAndEmptyArrays': True
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': '$release_index', 
                'releases_id': '$releases.id', 
                'releases_date': '$releases.date', 
                'releases_ocid': '$releases.ocid', 
                'parties': '$releases.parties', 
                'fecha_fallo': '$releases.tender.awardPeriod.endDate', 
                'codigo_expediente': '$releases.tender.id', 
                'parties': '$releases.parties'
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1, 
                'codigo_expediente': 1, 
                'numero_ganadores': {
                    '$size': {
                        '$filter': {
                            'input': '$parties', 
                            'as': 'e', 
                            'cond': {
                                '$eq': [
                                    '$$e.roles', [
                                        'tenderer', 'supplier'
                                    ]
                                ]
                            }
                        }
                    }
                }, 
                'numero_participantes': {
                    '$size': {
                        '$filter': {
                            'input': '$parties', 
                            'as': 'e', 
                            'cond': {
                                '$or': [
                                    {
                                        '$eq': [
                                            '$$e.roles', [
                                                'tenderer'
                                            ]
                                        ]
                                    }, {
                                        '$eq': [
                                            '$$e.roles', [
                                                'tenderer', 'supplier'
                                            ]
                                        ]
                                    }
                                ]
                            }
                        }
                    }
                },
                'participantes':{
                         '$filter': {
                            'input': '$parties', 
                            'as': 'e', 
                            'cond': {
                                '$or': [
                                    {
                                        '$eq': [
                                            '$$e.roles', [
                                                'tenderer'
                                            ]
                                        ]
                                    }, {
                                        '$eq': [
                                            '$$e.roles', [
                                                'tenderer', 'supplier'
                                            ]
                                        ]
                                    }
                                ]
                            }
                        }       
                },
                'ganadores': {
                    '$filter': {
                        'input': '$parties', 
                        'as': 'e', 
                        'cond': {
                            '$eq': [
                                '$$e.roles', [
                                    'tenderer', 'supplier'
                                ]
                            ]
                        }
                    }
                }, 
                'fecha_fallo': 1, 
                'parties': 1
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1, 
                'codigo_expediente': 1, 
                'numero_ganadores': 1, 
                'numero_participantes': 1,
                'nombre_participantes': {
                    '$filter': {
                        'input': '$participantes.name', 
                        'as': 'e', 
                        'cond': {
                            '$eq': [
                                1, 1
                            ]
                        }
                    }
                }, 
                'nombre_ganadores': {
                    '$filter': {
                        'input': '$ganadores.name', 
                        'as': 'e', 
                        'cond': {
                            '$eq': [
                                1, 1
                            ]
                        }
                    }
                }, 
                'fecha_fallo': 1, 
                'parties': 1
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1, 
                'codigo_expediente': 1, 
                'numero_ganadores': 1, 
                'numero_participantes': 1,
                'nombre_participantes': {
                    '$reduce': {
                        'input': '$nombre_participantes', 
                        'initialValue': '', 
                        'in': {
                            '$concat': [
                                '$$value', '$$this', ';'
                            ]
                        }
                    }
                },
                'nombre_ganadores': {
                    '$reduce': {
                        'input': '$nombre_ganadores', 
                        'initialValue': '', 
                        'in': {
                            '$concat': [
                                '$$value', '$$this', ';'
                            ]
                        }
                    }
                }, 
                'fecha_fallo': 1, 
                'parties': {
                    '$filter': {
                        'input': '$parties', 
                        'as': 'p', 
                        'cond': {
                            '$eq': [
                                '$$p.roles', [
                                    'procuringEntity'
                                ]
                            ]
                        }
                    }
                }
            }
        }, {
            '$unwind': {
                'path': '$parties', 
                'includeArrayIndex': 'parties_index', 
                'preserveNullAndEmptyArrays': True
            }
        }, {
            '$project': {
                'paquete': 1,
                '_id': 0,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1, 
                'codigo_expediente': 1, 
                'numero_ganadores': 1, 
                'numero_participantes': 1, 
                'nombre_participantes': 1,
                'nombre_ganadores': 1, 
                'fecha_fallo': 1, 
                'procuring_entity_country_name': '$parties.address.countryName', 
                'procuring_entity_locality': '$parties.address.locality', 
                'procuring_entity_postal_code': '$parties.address.postalCode', 
                'procuring_entity_region': '$parties.address.region', 
                'procuring_entity_street_addess': '$parties.address.streetAddress'
            }
        }
    ]

    return query_generals

## 1.2 Informacion de awards por items de la licitacion

In [6]:
def get_award_items(i: int)-> list:
    """
    Obtain the general pipeline query for i-th package
    """

    query_award_items = [
        {
            "$match": {"paquete": i}
        },
        {
            '$unwind': {
                'path': '$releases', 
                'includeArrayIndex': 'release_index', 
                'preserveNullAndEmptyArrays': True
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': '$release_index', 
                'releases_id': '$releases.id', 
                'releases_date': '$releases.date', 
                'releases_ocid': '$releases.ocid', 
                'codigo_expediente': '$releases.tender.id', 
                'parties': '$releases.parties', 
                'awards': '$releases.awards'
            }
        }, {
            '$unwind': {
                'path': '$awards', 
                'includeArrayIndex': 'award_index', 
                'preserveNullAndEmptyArrays': True
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1, 
                'codigo_expediente': 1,
                'parties': 1, 
                'awards_contract_period_enddate': '$awards.contractPeriod.endDate', 
                'awards_contract_period_startdate': '$awards.contractPeriod.startDate', 
                'award_description': '$awards.description', 
                'award_id': '$awards.id', 
                'award_items': '$awards.items', 
                'award_suppliers': '$awards.suppliers', 
                'award_title': '$awards.title', 
                'award_value_amount': '$awards.value.amount', 
                'award_value_currency': '$awards.value.currency'
            }
        }, {
            '$unwind': {
                'path': '$award_suppliers', 
                'includeArrayIndex': 'award_suppliers_index', 
                'preserveNullAndEmptyArrays': True
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1, 
                'codigo_expediente': 1,
                'parties': 1, 
                'awards_contract_period_enddate': 1, 
                'awards_contract_period_startdate': 1, 
                'award_description': 1, 
                'award_id': 1, 
                'award_items': 1, 
                'award_title': 1, 
                'award_value_amount': 1, 
                'award_value_currency': 1, 
                'award_suppliers_id': '$award_suppliers.id', 
                'award_suppliers_name': '$award_suppliers.name', 
                'award_suppliers_award_title': '$award_suppliers.award_title'
            }
        }, {
            '$unwind': {
                'path': '$award_items', 
                'includeArrayIndex': 'award_suppliers_items_index', 
                'preserveNullAndEmptyArrays': True
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1, 
                'codigo_expediente': 1,
                'parties': 1, 
                'awards_contract_period_enddate': 1, 
                'awards_contract_period_startdate': 1, 
                'award_description': 1, 
                'award_id': 1, 
                'award_title': 1, 
                'award_value_amount': 1, 
                'award_value_currency': 1, 
                'award_suppliers_id': 1, 
                'award_suppliers_name': 1, 
                'award_suppliers_award_title': 1, 
                'award_items_classification_description': '$award_items.classification.description', 
                'award_items_classification_id': '$award_items.classification.id', 
                'award_items_classification_scheme': '$award_items.classification.scheme', 
                'award_items_description': '$award_items.description', 
                'award_items_id': '$award_items.id', 
                'award_items_quantity': '$award_items.quantity', 
                'award_items_unit_name': '$award_items.unit.name', 
                'award_items_unit_value_amount': '$award_items.unit.value.amount', 
                'award_items_unit_value_currency': '$award_items.unit.value.currency', 
                'award_suppliers_items_index': 1
            }
        }, {
            '$unwind': {
                'path': '$parties', 
                'includeArrayIndex': 'parties_index', 
                'preserveNullAndEmptyArrays': True
            }
        }, {
            '$project': {
                'paquete': 1,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1,
                'codigo_expediente': 1, 
                'awards_contract_period_enddate': 1, 
                'awards_contract_period_startdate': 1, 
                'award_description': 1, 
                'award_id': 1, 
                'award_title': 1, 
                'award_value_amount': 1, 
                'award_value_currency': 1, 
                'award_suppliers_id': 1, 
                'award_suppliers_name': 1, 
                'award_suppliers_award_title': 1, 
                'award_items_classification_description': 1, 
                'award_items_classification_id': 1, 
                'award_items_classification_scheme': 1, 
                'award_items_description': 1, 
                'award_items_id': 1, 
                'award_items_quantity': 1, 
                'award_items_unit_name': 1, 
                'award_items_unit_value_amount': 1, 
                'award_items_unit_value_currency': 1, 
                'award_suppliers_items_index': 1, 
                'parties_index': 1,
                'parties_identifier_id':"$parties.identifier.id",
                'parties_name': "$parties.name", 
                'parties_address_country_name': '$parties.address.countryName', 
                'parties_address_locality': '$parties.address.locality', 
                'parties_address_postal_code': '$parties.address.postalCode', 
                'parties_address_region': '$parties.address.region', 
                'parties_address_street_address': '$parties.address.streetAddress', 
                'parties_roles': '$parties.roles'
            }
        }, {
            '$unwind': {
                'path': '$parties_roles', 
                'includeArrayIndex': 'parties_roles_index', 
                'preserveNullAndEmptyArrays': True
            }
        }, {
            '$match': {
                'parties_roles': 'supplier'
            }
        }, {
            '$project': {
                '_id': 0,
                'paquete': 1,
                'release_index': 1, 
                'releases_id': 1, 
                'releases_date': 1, 
                'releases_ocid': 1, 
                'codigo_expediente': 1,
                'awards_contract_period_enddate': 1, 
                'awards_contract_period_startdate': 1, 
                'award_description': 1, 
                'award_id': 1, 
                'award_title': 1, 
                'award_value_amount': 1, 
                'award_value_currency': 1, 
                'award_suppliers_id': 1, 
                'award_suppliers_name': 1, 
                'award_suppliers_award_title': 1, 
                'award_items_classification_description': 1, 
                'award_items_classification_id': 1, 
                'award_items_classification_scheme': 1, 
                'award_items_description': 1, 
                'award_items_id': 1, 
                'award_items_quantity': 1, 
                'award_items_unit_name': 1, 
                'award_items_unit_value_amount': 1, 
                'award_items_unit_value_currency': 1, 
                'award_suppliers_items_index': 1, 
                'supplier_index': '$parties_index',
                'supplier_id': "$parties_identifier_id",
                'supplier_name': "$parties_identifier_name",
                'supplier_address_country_name': '$parties_address_country_name', 
                'supplier_address_locality': '$parties_address_locality', 
                'supplier_address_postal_code': '$parties_address_postal_code', 
                'supplier_address_region': '$parties_address_region', 
                'supplier_address_street_address': '$parties_address_street_address', 
                'supplier_roles': '$parties.roles',
                'check': {'$eq': ['$award_suppliers_id','$parties_identifier_id']}
  
            }
        },
        {
        '$match': {'check': True}  
        }
    ]

    return query_award_items

### Generals Extraction

In [ ]:
for i in tqdm(range(1,99)):

    try:
        # creamos un cliente para interactuar con mongo
        myclient = pymongo.MongoClient(CONN_STRING)
        # conexion a la base edca.raw para colocar los json descargados
        db_raw = myclient["edca"]["raw"]

        generals_results = db_raw.aggregate(
            pipeline=get_query_generals(i)
            )
        
        myclient.close()

        generals = pd.DataFrame(generals_results)

        try:
            generals['releases_date'] = pd.to_datetime(generals['releases_date'])
        except:
            pass

        print(f"writing generals - paquete {i}")

        generals.to_csv(
            f'./processed/generals/generals_{i}.csv.zip',
            index=False,
            compression="zip"
            )

    except:
        print(f"Error on package {i}")
        pass

### Awards-Items Extraction

In [7]:
for i in tqdm(range(1,99)):
    try:

        # creamos un cliente para interactuar con mongo
        myclient = pymongo.MongoClient(CONN_STRING)
        # conexion a la base edca.raw para colocar los json descargados
        db_raw = myclient["edca"]["raw"]

        award_items_results = db_raw.aggregate(
                    pipeline=get_award_items(i)
                    )

        award_items = pd.DataFrame(award_items_results)

        myclient.close()

        try:
            award_items['releases_date'] = pd.to_datetime(award_items['releases_date'])
        except:
            pass

        try:
            award_items['awards_contract_period_enddate'] = pd.to_datetime(award_items['awards_contract_period_enddate'])
        except:
            pass

        try:
            award_items['awards_contract_period_startdate'] = pd.to_datetime(award_items['awards_contract_period_startdate'])
        except:
            pass

        award_items.fillna('', inplace=True)

        award_items['award_items_classification_id'] = award_items['award_items_classification_id'].astype(str)

        award_items['cucops_codigo_expediente'] = award_items.groupby(['codigo_expediente'])['award_items_classification_id'].transform(lambda x: '; '.join(list(set(x))))
        award_items['cucops_codigo_expediente_award_suppliers_id'] = award_items.groupby(['codigo_expediente', 'award_suppliers_id'])['award_items_classification_id'].transform(lambda x: '; '.join(list(set(x))))

        award_items.to_csv(f'./processed/award_items/award_items_{i}.csv.zip', index=False, compression='zip')

    except:
        print(f"Error on package {i}")
        pass

  0%|          | 0/98 [00:00<?, ?it/s]

### Edca Extraction

In [8]:
for k in tqdm(range(1,99)):
    try:
        file_awards_items = f"./processed/award_items/award_items_{k}.csv.zip"
        file_generals = f"./processed/generals/generals_{k}.csv.zip"
        
        awards_items = pd.read_csv(
            file_awards_items,
            compression='zip'
            )
        
        generals = pd.read_csv(
            file_generals,
            compression='zip'
            )

        edca = generals.merge(
            awards_items,
            how='left',
            on=[
            #'paquete',
            #'release_index',
            'releases_id',
            #'releases_date',
            #'releases_ocid'
            ]
            )
        
        edca.fillna('', inplace=True)

        edca.rename(columns={
            'paquete_x': 'paquete',
            'release_index_x': 'release_index',
            'releases_date_x': 'releases_date',
            'releases_ocid_x': 'releases_ocid'
            }, inplace=True
            )
        
        try:
            del edca['paquete_y']
        except:
            pass
        
        try:
            del edca['release_index_y']
        except:
            pass

        try:
            del edca['releases_date_y']
        except:
            pass

        try:
            del edca['releases_ocid_y']
        except:
            pass

        edca.to_csv(
            f"./processed/edca/edca_{k}.csv.zip",
            index=False,
            compression="zip"
            )

    except:
        print(f"Error on package {k}")
        pass

  0%|          | 0/98 [00:00<?, ?it/s]

/var/folders/xp/09_m9yj956x3kbv7qgpqgd6r0000gn/T/ipykernel_68330/3031636241.py:6: DtypeWarning: Columns (0,5,9,11,29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  awards_items = pd.read_csv(
/var/folders/xp/09_m9yj956x3kbv7qgpqgd6r0000gn/T/ipykernel_68330/3031636241.py:6: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  awards_items = pd.read_csv(
/var/folders/xp/09_m9yj956x3kbv7qgpqgd6r0000gn/T/ipykernel_68330/3031636241.py:6: DtypeWarning: Columns (29,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  awards_items = pd.read_csv(
/var/folders/xp/09_m9yj956x3kbv7qgpqgd6r0000gn/T/ipykernel_68330/3031636241.py:6: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  awards_items = pd.read_csv(
/var/folders/xp/09_m9yj956x3kbv7qgpqgd6r0000gn/T/ipykernel_68330/3031636241.py:6: DtypeWarning: Columns (29) have mixed types. Spe

In [ ]:
edca